## Imports

In [1]:
from enum import IntEnum

import numpy as np

from aoc_utilities import puzzle_input

<h2>--- Day 13: Transparent Origami ---</h2><p>You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.</p>
<p>Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:</p>
<pre><code>Congratulations on your purchase! To activate this infrared thermal imaging
camera system, please enter the code found on page 1 of the manual.
</code></pre>
<p>Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of <a href="https://en.wikipedia.org/wiki/Transparency_(projection)" target="_blank">transparent paper</a>! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:</p>
<pre><code>6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
</code></pre>
<p>The first section is a list of dots on the transparent paper. <code>0,0</code> represents the top-left coordinate.  The first value, <code>x</code>, increases to the right.  The second value, <code>y</code>, increases downward.  So, the coordinate <code>3,0</code> is to the right of <code>0,0</code>, and the coordinate <code>0,7</code> is below <code>0,0</code>. The coordinates in this example form the following pattern, where <code>#</code> is a dot on the paper and <code>.</code> is an empty, unmarked position:</p>
<pre><code>...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
</code></pre>
<p>Then, there is a list of <em>fold instructions</em>. Each instruction indicates a line on the transparent paper and wants you to fold the paper <em>up</em> (for horizontal <code>y=...</code> lines) or <em>left</em> (for vertical <code>x=...</code> lines). In this example, the first fold instruction is <code>fold along y=7</code>, which designates the line formed by all of the positions where <code>y</code> is <code>7</code> (marked here with <code>-</code>):</p>
<pre><code>...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
-----------
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
</code></pre>
<p>Because this is a horizontal line, fold the bottom half <em>up</em>. Some of the dots might end up overlapping after the fold is complete, but dots will never appear exactly on a fold line. The result of doing this fold looks like this:</p>
<pre><code>#.##..#..#.
#...#......
......#...#
#...#......
.#.#..#.###
...........
...........
</code></pre>
<p>Now, only <code>17</code> dots are visible.</p>
<p>Notice, for example, the two dots in the bottom left corner before the transparent paper is folded; after the fold is complete, those dots appear in the top left corner (at <code>0,0</code> and <code>0,1</code>). Because the paper is transparent, the dot just below them in the result (at <code>0,3</code>) remains visible, as it can be seen through the transparent paper.</p>
<p>Also notice that some dots can end up <em>overlapping</em>; in this case, the dots merge together and become a single dot.</p>
<p>The second fold instruction is <code>fold along x=5</code>, which indicates this line:</p>
<pre><code>#.##.|#..#.
#...#|.....
.....|#...#
#...#|.....
.#.#.|#.###
.....|.....
.....|.....
</code></pre>
<p>Because this is a vertical line, fold <em>left</em>:</p>
<pre><code>#####
#...#
#...#
#...#
#####
.....
.....
</code></pre>
<p>The instructions made a square!</p>
<p>The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, <code><em>17</em></code> dots are visible - dots that end up overlapping after the fold is completed count as a single dot.</p>
<p><em>How many dots are visible after completing just the first fold instruction on your transparent paper?</em></p>


## Solution

In [2]:
class Axis(IntEnum):
    X = 1
    Y = 0


def parse_transparency_dots_and_folds(origami_instructions):
    dots, fold_instructions = origami_instructions.strip().split('\n\n')
    dot_xs, dot_ys = (np.fromstring(dots.replace('\n', ','), sep=',', dtype=int)
                        .reshape(-1, 2)
                        .transpose())

    transparency = np.zeros((dot_ys.max() + 1, dot_xs.max() + 1), dtype=bool)
    transparency[(dot_ys, dot_xs)] = True

    fold_specifiers = (fold_instruction.rsplit(' ', maxsplit=1)[1].split('=')
                       for fold_instruction in fold_instructions.split('\n'))
    folds = [(getattr(Axis, name.upper()), int(coordinate))
             for name, coordinate in fold_specifiers]

    return transparency, folds


def fold(transparency, axis: Axis, coordinate):
    untouched_axis_slices = (axis % transparency.ndim) * [slice(None)]
    pre_fold_idx = tuple(untouched_axis_slices + [slice(coordinate)])
    post_fold_idx = tuple(untouched_axis_slices + [slice(None, coordinate, -1)])
    return transparency[pre_fold_idx] | transparency[post_fold_idx]

### Testing

In [3]:
example_transparency, example_folds = parse_transparency_dots_and_folds('''
    6,10
    0,14
    9,10
    0,3
    10,4
    4,11
    6,0
    6,12
    4,1
    0,13
    10,12
    3,4
    3,0
    8,4
    1,10
    2,14
    8,10
    9,0

    fold along y=7
    fold along x=5
''')

assert fold(example_transparency, *example_folds[0]).sum() == 17

### Answer

In [4]:
transparency, folds = parse_transparency_dots_and_folds(
    puzzle_input.as_text(day=13)
)

print(
    'Number of dots after the first fold:',
    fold(transparency, *folds[0]).sum()
)

Number of dots after the first fold: 671
